In [1]:
import pinecone
from tqdm import tqdm
from dotenv import load_dotenv
import os

# Load environment variables from .env file
#overide = true just forces a reload on the .env file in case api key changes
load_dotenv(override=True)

# Access the API key
pinecone_api_key = os.getenv("PINECONE_API_KEY")
huggingface_api_token = os.getenv("HUGGING_FACE_API_TOKEN")


C:\Users\Brady\.vscode\omnistudy-AI_Testing\venv\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
#Initialize the pinecone index
import pinecone      

pinecone.init(      
	api_key=pinecone_api_key,      
	environment='gcp-starter'      
)      
index = pinecone.Index('haystack')

#load a pinecone document store object with the index defined previously
from haystack.document_stores import PineconeDocumentStore

document_store = PineconeDocumentStore(
    api_key=pinecone_api_key,
    pinecone_index=index,
    similarity="cosine",
    embedding_dim=768
)

In [3]:
#Initialize retriever model
from haystack.nodes import EmbeddingRetriever
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
    model_format="sentence_transformers"
)
from haystack.pipelines import DocumentSearchPipeline

search_pipe = DocumentSearchPipeline(retriever=retriever)

C:\Users\Brady\.vscode\omnistudy-AI_Testing\venv\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
query = "How do you prepare for a coding interview?"

In [5]:
docs = search_pipe.run(query=query, params={"Retriever": {"top_k": 2}})['documents']
documents = []
for doc in docs:
    documents.append(doc.content)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
print("supporting documents provided \n")
for text in documents:
    print(text)

supporting documents provided 

Hints are provided at the back of this book, but push yourself to
develop a solution with as little help as possible. Many questions are designed to be tough-that's okay!
When you're solving a problem, make sure to think about the space and time efficiency.
2.
Write the code on paper. Coding on a computer offers luxuries such as syntax highlighting, code comple­
tion, and quick debugging. Coding on paper does not. Get used to this-and to how slow it is to write
and edit code-by coding on paper.
3. Test your code-on paper. This means testing the general cases, base cases, error cases, and so on. You'll
need to do this during your interview, so it's best to practice this in advance.
4. Type your paper code as-is into a computer. You will probably make a bunch of mistakes. Start a list of all
the errors you make so that you can keep these in mind during the actual interview.
In addition, try to do as many mock interviews as possible. You and a friend can ta

In [13]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "vblagoje/bart_lfqa"
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# model = model.to(device)


conditioned_doc = "<P> " + " <P> ".join([d for d in documents])
query_and_docs = "question: {} context: {}".format(query, conditioned_doc)

model_input = tokenizer(query_and_docs, truncation=True, padding=True, return_tensors="pt")

generated_answers_encoded = model.generate(input_ids=model_input["input_ids"],
                                           attention_mask=model_input["attention_mask"],
                                           min_length=64,
                                           max_length=256,
                                           do_sample=False, 
                                           early_stopping=True,
                                           num_beams=8,
                                           temperature=1.0,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
answer = tokenizer.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)


In [14]:
print(answer)

["I'm not sure if this is what you're looking for, but I'll give it a shot. First of all, you need to know what a coding interview is. It's not a job interview, it's an interview to see if you can be a good programmer. If you don't know how to code, you're not going to be able to do it well enough to get a good job. Second, you have to understand what the interviewer is looking for. They want you to be a programmer, but they don't want to hire someone who doesn't know what they're talking about. So, they ask you a bunch of questions, and you try to answer them as best as you can. You'll probably make a lot of mistakes during the interview, but you'll also learn a lot."]
